# Where are the events?
There are 3 sources:
 1. ipywidget.observe | click( handler_function( change) ) : [ipywidgets...Widget Events](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html)  
   Take advantage of change.old|new|name|owner|type event
 2. fig.canvas.mpl_connect( 'event_name', handler_function) : [matplotlib...event handling](https://matplotlib.org/3.2.2/users/event_handling.html)  
   Out of the box, the available events are:

    - 'button_press_event'
    - 'button_release_event'
    - 'draw_event'
    - 'key_press_event'
    - 'key_release_event'
    - 'motion_notify_event'
    - 'pick_event'
    - 'resize_event'
    - 'scroll_event'
    - 'figure_enter_event',
    - 'figure_leave_event',
    - 'axes_enter_event',
    - 'axes_leave_event'
    - 'close_event'  
    
    
 3. Navigation Toolbar2 : Not all possible events are implemented in [ipympl...backend_nbagg.py](https://github.com/matplotlib/ipympl/blob/master/ipympl/backend_nbagg.py) , so looking at [matplotlib...NavigationToolbar2](https://matplotlib.org/api/backend_bases_api.html#matplotlib.backend_bases.NavigationToolbar2) events, registered some essential toolbar missing events with mpl connect
    - 'home'
    - 'back',
    - 'forward'
    - 'zoom'
    - 'release_pan'
    - 'release_zoom'
    - 'drag_pan',
    - 'drag_zoom',  
    
Others I haven't tried: 'draw', 'draw_rubberband', 'mouse_move', 'pan', 'press', 'press_pan', 'press_zoom', 'push_current', 'release', 'remove_rubberband', 'save_figure', 'set_cursor', 'set_history_buttons', 'set_message', 'toolitems', 'update'  

Hacking in the home button pressed:  
```python
    home = Toolbar.home  
    def new_home(self, *args, **kwargs):  
        s = 'home_event'                     
        event = Event(s, self)  
        event.foo = 1                      
        self.canvas.callbacks.process(s, event)                        
        home(self, *args, **kwargs)
    Toolbar.home = new_home            
```


 

# Click & Move event (Inside Axes printing in a native text box)

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

In [2]:
fig, ax = plt.subplots()
tb=fig.text(x=0.6,y=0.9,s='place holder')
plt.show()

In [3]:
state={ 'pressId':False, 'releaseId':False, 'motionId':False, 'lastx':False, 'lasty':False}

def motion_handler(event):
    if state['lastx']!=False:
        mouse_travel = np.linalg.norm( [ state['lastx'] - event.x, state['lasty'] - event.y ] )
        if mouse_travel > 10:
            if event.button==1:
                print('button 1')
                tb.set_text('Button:1, m.travel:{:.3}'.format(mouse_travel))
            if event.button==3:
                print('button 3')
                tb.set_text('Button:3, m.travel:{:.3}'.format(mouse_travel))
    state['lastx']=event.x
    state['lasty']=event.y
    event.canvas.draw()
        
def button_press_handler(event):
    print('PRESS')
    state['motionId']  = fig.canvas.mpl_connect('motion_notify_event'  , motion_handler)
    state['releaseId'] = fig.canvas.mpl_connect('button_release_event' , button_release_handler)
    
def button_release_handler(event):
    print('RELEASE')
    fig.canvas.mpl_disconnect(state['motionId'])
    
def axes_enter_handler(event):
    print('ENTER')
    tb.set_text('Press left or right mouse button')
    state['pressId'] = fig.canvas.mpl_connect('button_press_event'  , button_press_handler)
    event.canvas.draw()

def axes_leave_handler(event):
    print('LEAVE')
    tb.set_text('Left...')
    fig.canvas.mpl_disconnect(state['pressId'])
    fig.canvas.mpl_disconnect(state['releaseId'])
    fig.canvas.mpl_disconnect(state['motionId'])
    event.canvas.draw()
    
fig.canvas.mpl_connect('axes_enter_event',axes_enter_handler)
fig.canvas.mpl_connect('axes_leave_event',axes_leave_handler)

9

ENTER
PRESS
button 1
button 1
button 1
button 1
RELEASE
LEAVE
ENTER
PRESS
button 3
button 3
button 3
RELEASE
LEAVE
ENTER
PRESS
button 1
button 1
button 1
button 1
button 1
button 1
button 1
button 1
button 1
button 1
button 1
RELEASE
LEAVE


## get clicks on figure
https://matplotlib.org/3.2.2/users/event_handling.html  
Don't forget prints are seen on the Log Console (View > Show Log Console), enhancing to INFO or DEBUG level

In [ ]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
fig, ax = plt.subplots()
fig.canvas.capture_scroll = False
ax.plot(np.random.rand(10))

def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%.3f, ydata=%.3f, name=%s' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata, event.name))

def ondraw(event):
    # print 'ondraw', event
    # these ax.limits can be stored and reused as-is for set_xlim/set_ylim later
    print('redraw xlim=%.3f,%.3f ylim=%.3f,%.3f' % (ax.get_xlim()[0], ax.get_xlim()[1], 
                                                    ax.get_ylim()[0], ax.get_ylim()[1]) )

def onbutton(event):
    print('button_press_event', event.button, event.name, event.x, event.y, event.xdata, event.ydata, event.name)
        
cidPress   = fig.canvas.mpl_connect('button_press_event'  , onclick)
cidDelease = fig.canvas.mpl_connect('button_release_event', onclick)
cidDraw    = fig.canvas.mpl_connect('draw_event', ondraw)
cidBP      = fig.canvas.mpl_connect('button_press_event', onbutton)

plt.show()

## get clicks on toolbar
https://stackoverflow.com/questions/14896580/matplotlib-hooking-in-to-home-back-forward-button-events

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backend_bases import NavigationToolbar2

home = NavigationToolbar2.home

def new_home(self, *args, **kwargs):
    print 'new home'
    home(self, *args, **kwargs)

NavigationToolbar2.home = new_home

fig = plt.figure()
plt.text(0.35, 0.5, 'Hello world!', dict(size=30))
plt.show()

In [ ]:
from matplotlib.backend_bases import NavigationToolbar2, Event

home = NavigationToolbar2.home

def new_home(self, *args, **kwargs):
    s = 'home_event'
    event = Event(s, self)
    event.foo = 100
    self.canvas.callbacks.process(s, event)
    home(self, *args, **kwargs)

NavigationToolbar2.home = new_home

def handle_home(evt):
    print('new home')
    print(evt.foo)

fig = plt.figure()
fig.canvas.mpl_connect('home_event', handle_home)
plt.text(0.35, 0.5, 'Hello world!', dict(size=30))
plt.show()

### Now using ipympl

In [ ]:
from matplotlib.backend_bases import Event
from ipympl.backend_nbagg import Toolbar

#HOME
home = Toolbar.home

def new_home(self, *args, **kwargs):
    s = 'home_event'
    event = Event(s, self)
    event.foo = 0
    self.canvas.callbacks.process(s, event)
    home(self, *args, **kwargs)

Toolbar.home = new_home

def handle_home(evt):
    print('new home ',evt.foo)

#ZOOM
zoom = Toolbar.zoom

def new_zoom(self, *args, **kwargs):
    s = 'zoom_event'
    event = Event(s, self)
    event.foo = 1
    self.canvas.callbacks.process(s, event)
    zoom(self, *args, **kwargs)

Toolbar.zoom = new_zoom

def handle_zoom(evt):
    print('new zoom ' , evt.foo, ax.get_xlim(), ax.get_ylim())

#PRESS
press_zoom = Toolbar.press_zoom

def new_press_zoom(self, *args, **kwargs):
    s = 'press_zoom_event'
    event = Event(s, self)
    event.foo = 2
    self.canvas.callbacks.process(s, event)
    press_zoom(self, *args, **kwargs)

Toolbar.press_zoom = new_press_zoom

def handle_press_zoom(evt):
    print('new press_zoom ' , evt.foo, ax.get_xlim(), ax.get_ylim())

#release
release_zoom = Toolbar.release_zoom

def new_release_zoom(self, *args, **kwargs):
    s = 'release_zoom_event'
    event = Event(s, self)
    event.foo = 3
    self.canvas.callbacks.process(s, event)
    release_zoom(self, *args, **kwargs)

Toolbar.release_zoom = new_release_zoom

def handle_release_zoom(evt):
    print('new release_zoom ' , evt.foo, ax.get_xlim(), ax.get_ylim())

def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%.3f, ydata=%.3f, name=%s' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata, event.name))    

fig, ax = plt.subplots()

fig.canvas.mpl_connect('home_event', handle_home)
fig.canvas.mpl_connect('zoom_event', handle_zoom)
fig.canvas.mpl_connect('press_zoom_event', handle_press_zoom)
fig.canvas.mpl_connect('release_zoom_event', handle_release_zoom)
cidPress   = fig.canvas.mpl_connect('button_press_event'  , onclick)
cidDelease = fig.canvas.mpl_connect('button_release_event', onclick)

ax.plot(np.random.rand(10))
plt.text(0.35, 0.5, 'Hello world!', dict(size=30))
plt.show()

### get zoom to what? draw event

In [ ]:
from matplotlib import pyplot as plt
import numpy

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(numpy.random.rand(10))

def ondraw(event):
    # print 'ondraw', event
    # these ax.limits can be stored and reused as-is for set_xlim/set_ylim later
    print(ax.get_xlim(), ax.get_ylim())

cid = fig.canvas.mpl_connect('draw_event', ondraw)

plt.show()

In [ ]:
from matplotlib.backend_bases import NavigationToolbar2 as t
from matplotlib.backends.backend_webagg_core import NavigationToolbar2WebAgg as t2
display(dir(t),dir(t2))

# Connect ~all~ most of the widget toolbar

In [3]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.backend_bases import Event
from ipympl.backend_nbagg import Toolbar

#HOME
home = Toolbar.home

def new_home(self, *args, **kwargs):
    s = 'home_event'
    event = Event(s, self)
    event.foo = 0
    self.canvas.callbacks.process(s, event)
    home(self, *args, **kwargs)

Toolbar.home = new_home

def handle_home(evt):
    print('new home ',evt.foo)

#ZOOM
zoom = Toolbar.zoom

def new_zoom(self, *args, **kwargs):
    s = 'zoom_event'
    event = Event(s, self)
    event.foo = 1
    self.canvas.callbacks.process(s, event)
    zoom(self, *args, **kwargs)

Toolbar.zoom = new_zoom

def handle_zoom(evt):
    print('new zoom ' , evt.foo, ax.get_xlim(), ax.get_ylim())

#PRESS
press_zoom = Toolbar.press_zoom

def new_press_zoom(self, *args, **kwargs):
    s = 'press_zoom_event'
    event = Event(s, self)
    event.foo = 2
    self.canvas.callbacks.process(s, event)
    press_zoom(self, *args, **kwargs)

Toolbar.press_zoom = new_press_zoom

def handle_press_zoom(evt):
    print('new press_zoom ' , evt.foo, ax.get_xlim(), ax.get_ylim())

#release
release_zoom = Toolbar.release_zoom

def new_release_zoom(self, *args, **kwargs):
    s = 'release_zoom_event'
    event = Event(s, self)
    event.foo = 3
    self.canvas.callbacks.process(s, event)
    release_zoom(self, *args, **kwargs)

Toolbar.release_zoom = new_release_zoom

def handle_release_zoom(evt):
    print('new release_zoom ' , evt.foo, ax.get_xlim(), ax.get_ylim())

def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%.3f, ydata=%.3f, name=%s' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata, event.name))    

fig, ax = plt.subplots()

fig.canvas.mpl_connect('home_event', handle_home)
fig.canvas.mpl_connect('zoom_event', handle_zoom)
fig.canvas.mpl_connect('press_zoom_event', handle_press_zoom)
fig.canvas.mpl_connect('release_zoom_event', handle_release_zoom)
cidPress   = fig.canvas.mpl_connect('button_press_event'  , onclick)
cidDelease = fig.canvas.mpl_connect('button_release_event', onclick)
import numpy as np
ax.plot(np.random.rand(10))
plt.text(0.35, 0.5, 'Hello world!', dict(size=30))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [2]:
import matplotlib_ipywidgets_events as mwe

## Showcasing connected toolbar to events

In [3]:
a = mwe.App1()
a

App1(children=(VBox(children=(Label(value='Hello Title!', layout=Layout(align_self='center')), IntSlider(value…

## Showcasing pick data event

In [4]:
b=mwe.App2()
b

App2(children=(VBox(children=(Label(value='El título es', layout=Layout(align_self='center')), IntSlider(value…